### 3Dの対応点から3Dの変換パラメータ（6つ)を算出する。

#### 推定前
<img src="ipynb_png/3d-3dCorrespondence_bunny_before.png">

#### 推定後
<img src="ipynb_png/3d-3dCorrespondence_bunny_after.png">


In [1]:
from pyntcloud import PyntCloud
import numpy as np
import pandas as pd
import cv2
import math

In [2]:
#同次座標系に関する関数
def makeHomogeneous(vector):
    rows,cols = vector.shape[:2]
    ones = [np.ones(cols)]
    return np.r_[vector, ones]
    
def delHomogeneous(vector):
    rows,cols = vector.shape[:2]
    val = vector[:rows-1]
    dim = vector[rows-1:]
    return val / dim

def sampling3d(point3d, samples):
    indexes = np.random.randint(0, point3d.shape[0], samples)
    return point3d[indexes]

#三次元回転行列生成に関する関数
    # 物体座標系の 1->2->3 軸で回転させる
def generateRotationMatrix(r):
    px = r[0]
    py = r[1]
    pz = r[2]
    
    Rx = np.array([[1, 0, 0],
            [0, np.cos(px), np.sin(px)],
            [0, -np.sin(px), np.cos(px)]])
    Ry = np.array([[np.cos(py), 0, -np.sin(py)],
            [0, 1, 0],
            [np.sin(py), 0, np.cos(py)]])
    Rz = np.array([[np.cos(pz), np.sin(pz), 0],
            [-np.sin(pz), np.cos(pz), 0],
            [0, 0, 1]])
    R = Rz.dot(Ry).dot(Rx)
    return R

def generateTransMatrix(t, r):
    R = generateRotationMatrix(r)
    transMatrix = np.eye(4,4)
    for y in range(0,3):
        for x in range(0,3):
            transMatrix[y, x] = R[y, x]
        transMatrix[y, 3] = t[y]
    return transMatrix

def generatePyntCloud(point3d):
    """numpy形式の3d点群からpyntcloud用データを生成する。
       @param[in] point3d [[x,y,z],...] 
    """
    w_color = False
    if(point3d.shape[1] == 6):
        w_color = True
    points = pd.DataFrame(point3d)

    if(w_color == False):
        points.columns = ['x', 'y', 'z']
    else:
        points.columns = ['x', 'y', 'z', 'blue', 'green', 'red']
    cloud = PyntCloud(points)
    return cloud

def axisPyntCloud(scale):
    # color, [vertexes]
    lines = {
        '0xFF0000': [[0, 0, 0], [scale, 0, 0]],
        '0x00FF00': [[0, 0, 0], [0, scale, 0]],
        '0x0000FF': [[0, 0, 0], [0, 0, scale]]
    }
    return lines

def addColor(point3d, color):
    return np.concatenate((point3d, np.ones(point3d.shape) * np.array(color)) ,axis=1)

def warp3d(point3d, M):
    h_point3d = makeHomogeneous(point3d)
    h_w_point3d = M.dot(h_point3d)
    return delHomogeneous(h_w_point3d)

In [3]:
#bunnyのデータを読み込み、重心中心に移動する
cloud = PyntCloud.from_file("data/stanford/bun_zipper.ply")
point3d = sampling3d(cloud.get_mesh_vertices()[0], 1000)
point3d = point3d - np.mean(point3d, axis = 0)
cloud = generatePyntCloud(addColor(point3d, [0, 255, 255]))
cloud.plot(point_size=0.002, opacity=1.0, polylines=axisPyntCloud(0.1))

In [4]:
#モデルデータ生成
obj = point3d.T

#観測データ生成
#位置姿勢の初期値（ランダム）
t = np.random.randn(3)*0.03
p = np.random.randn(3) * np.pi / 16
dM = generateTransMatrix(t, p)

obs = warp3d(obj, dM)

cloud = generatePyntCloud(np.concatenate((addColor(obs.T, [0, 255, 255]), addColor(obj.T,[0, 0, 255]))))
cloud.plot(point_size=0.002, opacity=1.0, polylines=axisPyntCloud(0.1))

### ヤコビアン計算

$X_0 = (x_0, y_0, z_0)$を$X_1 = (x_1, y_1, z_1)$に変換するとき、  
$$
e = \frac{1}{2} \Sigma (X_0 - X_1)^2
$$
を最小化するような幾何変換$ \Delta R, \Delta t$を最小二乗法により算出する。  
変換式は
$$ 
\begin{eqnarray}
  \left(
    \begin{array}{c}
      x_{1} \\
      y_{1}
    \end{array}
  \right)
  &=&
  \Delta R
  \left(
    \begin{array}{c}
      x_{0}\\
      y_{0}\\
      z_{0}
    \end{array}
  \right)
  +
  \Delta t
  \\
  &=&
  \left(
    \begin{array}{c}
      1 & - \Delta \omega_z & \Delta \omega_y\\
      \Delta \omega_z & 1 & - \Delta \omega_z\\
      - \Delta \omega_y & \Delta \omega_x & 1\\
    \end{array}
  \right)
  \left(
    \begin{array}{c}
      x_{0}\\
      y_{0}\\
      z_{0}
    \end{array}
  \right)
  +
  \left(
    \begin{array}{c}
      \Delta t_{x}\\
      \Delta t_{y}\\
      \Delta t_{z}
    \end{array}
  \right)
  \end{eqnarray}
$$ 

このとき、ヤコビ行列$J$は
$$ 
\begin{eqnarray}
  J
  &=&
  \left(
    \begin{array}{c,c,c}
      \frac{\delta x}{\delta t_{x}} &
      \frac{\delta x}{\delta t_{y}} &
      \frac{\delta x}{\delta t_{z}} &
      \frac{\delta x}{\delta \omega_x} &
      \frac{\delta x}{\delta \omega_y} &
      \frac{\delta x}{\delta \omega_z} \\
      \frac{\delta y}{\delta t_{x}} &
      \frac{\delta y}{\delta t_{y}} &
      \frac{\delta y}{\delta t_{z}} &
      \frac{\delta y}{\delta \omega_x} &
      \frac{\delta y}{\delta \omega_y} &
      \frac{\delta y}{\delta \omega_z} \\
      \frac{\delta z}{\delta t_{x}} &
      \frac{\delta z}{\delta t_{y}} &
      \frac{\delta z}{\delta t_{z}} &
      \frac{\delta z}{\delta \omega_x} &
      \frac{\delta z}{\delta \omega_y} &
      \frac{\delta z}{\delta \omega_z}
    \end{array}
  \right)
  \\
  &=&
  \left(
    \begin{array}{c,c,c}
    1 & 0 & 0 & 0 &  z_0 & - y_0\\
    0 & 1 & 0 & -z_0 & 0 & x_0\\
    0 & 0 & 1 & y_0 & - x_0 & 0\\
    \end{array}
  \right)
  \end{eqnarray}
$$ 
となる。  
ところで、各パラメータによる微分が0(極値)となる条件は$\Delta E = 0$である。
つまり、
$$
J^tJ \Delta = J^Te 
$$
$$
\Delta = (J^tJ)^{-1}J^Te 
$$
である。これを解く各種solverは多数ライブラリが存在しており、一般的には$J^tJ$と$J^Te$を引数として取る。そこで、繰り返し計算により$\Delta R, \Delta t$を計算し$J^tJ$と$J^Te$を更新し、徐々に誤差を小さくする。

In [5]:
#ヤコビアン計算
def calcJacob(obj, obs, deltaPose, JtJ, JtE):
    if obj.shape[1] != obs.shape[1]:
        print ("ERROR data size is not the same!")
    #座標変換
    #obj_h = makeHomogeneous(obj)
    #obs_h = deltaPose.dot(obj_h)
    #est = delHomogeneous(obs_h)
    est = obj
    
    for (p, q) in zip(est.transpose(), obs.transpose()):
        X = p[0]
        Y = p[1]
        Z = p[2]
        ex = q[0] - p[0]
        ey = q[1] - p[1]
        ez = q[2] - p[2]
        
        JtJ[0,0] += 1.0
        JtJ[0,4] += Z
        JtJ[0,5] += -Y
        JtJ[1,1] += 1.0
        JtJ[1,3] += -Z
        JtJ[1,5] += X
        JtJ[2,2] += 1.0
        JtJ[2,3] += Y
        JtJ[2,4] += -Z
        JtJ[3,3] += Y*Y + Z*Z
        JtJ[3,4] += -X * Y
        JtJ[3,5] += -X * Z
        JtJ[4,4] += X*X + Z*Z
        JtJ[4,5] += -Y * Z
        JtJ[5,5] += X*X + Y*Y
        
        JtE[0,0] += ex
        JtE[1,0] += ey
        JtE[2,0] += ez
        JtE[3,0] += -Z * ey + Y * ez
        JtE[4,0] += Z * ex - X * ez
        JtE[5,0] += -Y * ex + X * ey
    
    #fill
    JtJ[1,0] = JtJ[0,1]
    JtJ[2,0] = JtJ[0,2]
    JtJ[2,1] = JtJ[1,2]
    JtJ[3,0] = JtJ[0,3]
    JtJ[3,1] = JtJ[1,3]
    JtJ[3,2] = JtJ[2,3]
    JtJ[4,0] = JtJ[0,4]
    JtJ[4,1] = JtJ[1,4]
    JtJ[4,2] = JtJ[2,4]
    JtJ[4,3] = JtJ[3,4]
    JtJ[5,0] = JtJ[0,5]
    JtJ[5,1] = JtJ[1,5]
    JtJ[5,2] = JtJ[2,5]
    JtJ[5,3] = JtJ[3,5]
    JtJ[5,4] = JtJ[4,5]

In [6]:
def calcPose3D(obj, obs, deltaPose):
    #ヤコビ行列生成
    JtJ = np.zeros((6,6))
    JtE = np.zeros((6,1))
    calcJacob(obj, obs, deltaPose, JtJ, JtE)
    
    #solve
    x = np.linalg.solve(JtJ, JtE)
    t = x[0:3]
    omega = x[3:6]

    #補正量計算
    angle = math.sqrt(omega[0]*omega[0] + omega[1]*omega[1]+omega[2]*omega[2])
    dR = np.eye(3,3)
    if angle < 1.0e-12:
        print ("rot small")
    else:
        dR[0,1] = -omega[2]
        dR[0,2] = omega[1]
        dR[1,0] = omega[2]
        dR[1,2] = -omega[0]
        dR[2,0] = -omega[1]
        dR[2,1] = omega[0]
    
    #行列生成
    transMatrix = np.eye(4,4)
    for y in range(0,3):
        for x in range(0,3):
            transMatrix[y, x] = dR[y, x]
        transMatrix[y, 3] = t[y]
    return transMatrix

In [7]:
estPose = np.eye(4,4)
for i in range(10):
    w_obj = warp3d(obj, estPose)
    deltaPose = calcPose3D(w_obj, obs, estPose)
    estPose = deltaPose.dot(estPose)
print(estPose)
print(dM)

[[ 0.9999296  -0.08441962  0.05557668  0.0048316 ]
 [ 0.08447401  0.99991779 -0.06714666 -0.06107164]
 [-0.05549569  0.06721192  0.99994437  0.01360351]
 [ 0.          0.          0.          1.        ]]
[[ 0.99493303 -0.08162085  0.05870524  0.0048316 ]
 [ 0.08535099  0.9942857  -0.06411823 -0.06107164]
 [-0.05313639  0.06880389  0.99621411  0.01360351]
 [ 0.          0.          0.          1.        ]]


In [8]:
est = warp3d(obj, estPose)
cloud = generatePyntCloud(np.concatenate((addColor(est.T, [0, 255, 255]), addColor(obs.T,[0, 0, 255]))))
cloud.plot(point_size=0.002, opacity=1.0, polylines=axisPyntCloud(0.1))